In [ ]:
!pip install pycryptodome
!pip install argon2_cffi
!pip install python-secrets

In [16]:
from Crypto.Cipher import AES
from Crypto.Util import Counter
from Crypto import Random
import binascii
import argon2
import secrets
import os

# AES supports multiple key sizes: 16 (AES128), 24 (AES192), or 32 (AES256).
key_bytes = 32

In [3]:
# Takes as input a 32-byte key and an arbitrary-length plaintext and returns a
# pair (iv, ciphtertext). "iv" stands for initialization vector.
def encrypt(key, plaintext):
    assert len(key) == key_bytes

    # Choose a random, 16-byte IV.
    iv = Random.new().read(AES.block_size)

    # Convert the IV to a Python integer.
    iv_int = int(binascii.hexlify(iv), 16) 

    # Create a new Counter object with IV = iv_int.
    ctr = Counter.new(AES.block_size * 8, initial_value=iv_int)

    # Create AES-CTR cipher.
    aes = AES.new(key, AES.MODE_CTR, counter=ctr)

    # Encrypt and return IV and ciphertext.
    ciphertext = aes.encrypt(plaintext)
    return (iv, ciphertext)

In [4]:
# Takes as input a 32-byte key, a 16-byte IV, and a ciphertext, and outputs the
# corresponding plaintext.
def decrypt(key, iv, ciphertext):
    assert len(key) == key_bytes

    # Initialize counter for decryption. iv should be the same as the output of
    # encrypt().
    #iv_int = int(iv.encode('hex'), 16) 
    iv_int = int(binascii.hexlify(iv), 16) 
    ctr = Counter.new(AES.block_size * 8, initial_value=iv_int)

    # Create AES-CTR cipher.
    aes = AES.new(key, AES.MODE_CTR, counter=ctr)

    # Decrypt and return the plaintext.
    plaintext = aes.decrypt(ciphertext)
    return plaintext

In [5]:
def salt_generator():
    #generating salt for better entropy
    #industry recommended standard is to use CSPRNG (Cryptographically Secure Pseudo-Random Number Generator) to produce a salt.

    return secrets.token_bytes(32)

In [6]:
def validation(password):
    #block to generate hash to validate whether the password is correct or not 

    argon2Hasher = argon2.PasswordHasher(time_cost=16, memory_cost=2**15, parallelism=2, hash_len=32, salt_len=16)
    hash = argon2Hasher.hash(password)

    return  hash

In [10]:
def check_password(hash, password):
    #to check whether entered subject(password) by the user is correct or not
    argon2Hasher = argon2.PasswordHasher(time_cost=16, memory_cost=2**15, parallelism=2, hash_len=32, salt_len=16)
       
    try:
        return argon2Hasher.verify(hash, password)
         
    except:
        return False

In [9]:
def key_generator_argon2(password, salt):
    #argon2 is industry specified hashing algorithm. Winner of Password Hashing Competition

    password = password.encode('utf-8')

    hash = argon2.hash_password_raw(time_cost=16, memory_cost=2**15, parallelism=2, hash_len=32,
    password=password, salt=salt, type=argon2.low_level.Type.ID)

    return hash


In [12]:
def read_file(doc):

    a = doc.read()

    name = doc.name
    doc.close()
    if os.path.exists(name):
       os.remove(name)

    return a

In [13]:
def encryption(document, password):

    text_to_encrypt = read_file(document)
    text_to_encrypt = text_to_encrypt.encode('utf-8')
    salt = salt_generator()
    key = key_generator_argon2(password, salt)

    hashed_password = validation(password)

    (iv, ciphertext) = encrypt(key, text_to_encrypt)

    return salt, iv, hashed_password, ciphertext

In [14]:
def decryption(salt, iv, ciphertext, password):

    key = key_generator_argon2(password, salt)

    decrypted_data = decrypt(key, iv, ciphertext)

    return decrypted_data

In [18]:
password = "mynameisapples"

doc = open("enc.txt","r")

(salt, iv, hashed_password, ciphertext) = encryption(doc, password)

print(salt)
print(iv)
print(hashed_password)
print(ciphertext)

decrypted_data = decryption(salt, iv, ciphertext, password)

print(decrypted_data)

b'\x13\xc9\x8du\xc9\x02\x1e{\xc7\x13\xc4\xd9\x1cz\x94n\xd7\x80\x07\xe2O]>\x98\xbb\x82(N%Fc\xb6'
b'\xbb*\x99\xc1\xc4#\xa4\xc3@\xbf\xbe\x99H\x14\xd6\x18'
$argon2id$v=19$m=32768,t=16,p=2$4SefMNR/983SiSJ/Nbbcwg$auf8S/1cHeBo7j0vOtEJuSo4ZJ4L5UdM9swIgWBoaZ4
b'\xb6\x02!6J\xc4\xd5\x06D\xf79\xf9\x99\xa7\x976=\xda\xb7\xa3\xac\x02-\x12l\x9f\xd6\xefW"\xa6\x16\xaa@\xdfm\x1dON\x85\t\xaa7\x94\xfb\xb6k\x95?'
b'Hello this is a document which will be encrypted.'
